# Motivation

This is a clean version of the experiments I performed for the CN Poster and Extended abstract. Some relatable notebooks are:

* **SBM_h42_TF_network_effects** - where i am looking at selective edge pruning
* **SBM_allHelathy_gc42** - where i'm analysing the all_healthy data

This is a modified version of the notebook CN_SBM_allHealty.ipynb from iNET repository.

# Init

In [2]:
%load_ext autoreload
import pandas as pd
import numpy as np
import os
import sys
import igraph as ig

import plotly.express as px


import multiprocess as mp


# own libraries
SCRIPT_DIR = os.path.dirname(os.path.abspath("pcgna_processing.py"))
sys.path.append(os.path.dirname(SCRIPT_DIR))
sys.path.append('/Users/vlad/Documents/Code/York/iNet_v2/src/')

from NetworkAnalysis.ExperimentSet import ExperimentSet
from NetworkAnalysis import GraphHelper as gh
from NetworkAnalysis.utilities import sankey_consensus_plot as sky
from NetworkAnalysis.utilities.helpers import save_fig
from NetworkAnalysis.utilities.helpers import save_fig, survival_plot, survival_comp
from NetworkAnalysis.utilities.Graph_Tool.GraphToolExp import GraphToolExperiment as GtExp
sys.path.append(os.path.dirname("../../src"))
# Gsea libraries

import graph_tool.all as gt

# matplotlib.use("gtk3agg")

# %matplotlib inline
import gseapy as gp

pool = mp.Pool(mp.cpu_count())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
results_path = "../../results/exp/"
data_base = "../../data/"
base_path = "../../results/"
exp_folder_tumour = "network_I/tum/" # "/integration_v2.1/ - path from iNET
exp_folder_h42 = "network_I/healthy42/"
exp_folder_h42_ctrl = "network_I/healthyControls/"

figures_path = "selective_edge_pruning/"

vu_output = pd.read_csv(f"{data_base}/metadata/VU_clustering_v3.tsv", sep="\t", index_col="Sample")

tcga_mutations_df = pd.read_csv(f"{data_base}/tumour/mutations_tcga.csv")
tcga_mutations_df = tcga_mutations_df[tcga_mutations_df["count"] != 0].set_index("gene")

all_tum_tpms = pd.read_csv(f"{data_base}/tumour/TPMs_selected_genes_v3_13k_gc42.tsv", sep="\t", index_col="genes")

# tf list
tf_path = f"{data_base}/metadata/TF_names_v_1.01.txt"
if os.path.exists(tf_path):
    tf_list = np.genfromtxt(fname=tf_path, delimiter="\t", skip_header=1, dtype="str")

## Clean GraphML

In [4]:
# FOR SBM network
if False:
    test_graph = ig.read(filename=figures_path + "SBM_5K_6TF.graphml")

    for vertex in test_graph.vs:
        vertex["tf"] = bool(vertex["tf"])
        vertex["ctrl_tf"] = bool(vertex["ctrl_tf"])
        vertex["mut_count"] = int(vertex["count"])
        vertex["ivi"] = round(vertex["ivi"])
        vertex["Leiden_TF_6"] = str(vertex["modularity class"])
        vertex["DC-SBM_TF_6"] = str(vertex["sbm_0"])
        vertex["SBM_stable"] = bool(vertex["smb_stable_0_num"])

    del test_graph.vs["modularity class"]
    del test_graph.vs["sbm_0"]
    del test_graph.vs["smb_stable_0_num"]
    del test_graph.vs["lund"]
    del test_graph.vs["tcga"]
    del test_graph.vs["count"]

    test_graph.write_graphml(figures_path + "SBM_5K_6TF_v2.graphml")

    # For Leiden
    test_graph = ig.read(filename=figures_path + "Leiden_5K_50TF.graphml")

    for vertex in test_graph.vs:
        vertex["tf"] = bool(vertex["tf"])
        vertex["ctrl_tf"] = bool(vertex["ctrl_tf"])
        vertex["mut_count"] = int(vertex["count"])
        vertex["ivi"] = round(vertex["ivi"])
        vertex["Leiden_TF_50"] = str(vertex["modularity class"])

    del test_graph.vs["modularity class"]
    del test_graph.vs["lund"]
    del test_graph.vs["tcga"]
    del test_graph.vs["count"]

    test_graph.write_graphml(figures_path + "Leiden_5K_50TF_v2.graphml")

## Load experiment sets

In [5]:
%autoreload 2
all_h = ExperimentSet("allHealthy42", base_path, exp_folder_h42, tcga_mutations_df, sel_sets = ["5K"], rel_path="../")
all_h.export_to_gephi(save=False)

tf_range = [3, 6, 10]
exps, entropy = GtExp.load_sbm_exps(all_h, name="standard_5K", exp_type="h42_700_degreeCor", tf_range=tf_range)

##### Experiment labels:  dict_keys(['standard_5K_3TF', 'norm3_5K_3TF', 'beta_5K_3TF', 'standard_5K_6TF', 'standard_5K_10TF', 'norm3_5K_6TF', 'beta_5K_6TF', 'norm3_5K_10TF', 'beta_5K_10TF', 'standard_5K_4TF', 'standard_5K_7TF', 'standard_5K_8TF', 'standard_5K_5TF', 'standard_5K_9TF', 'norm3_5K_4TF', 'beta_5K_4TF', 'standard_5K_15TF', 'standard_5K_20TF', 'standard_5K_30TF', 'standard_5K_25TF', 'standard_5K_40TF', 'standard_5K_35TF', 'standard_5K_45TF', 'standard_5K_50TF', 'standard_5K_12TF', 'standard_5K_14TF', 'standard_5K_11TF', 'standard_5K_13TF'])
### Loaded standard_int_healthy42_5K_3TF
### Loaded standard_int_healthy42_5K_6TF
### Loaded standard_int_healthy42_5K_10TF


In [6]:
%%capture
folders = next(os.walk(base_path + exp_folder_h42_ctrl), (None, None, []))[1]
args, h_ctrls = [], {}
for folder in folders:
    hCtrl_path = "{}/{}/".format(exp_folder_h42_ctrl, folder)
    idx = int(folder.split("hCtrl")[-1])
    h_ctrls[idx] = ExperimentSet("allHealthy42", base_path, hCtrl_path, tcga_mutations_df, sel_sets=["5K"], rel_path="../")
    h_ctrls[idx].export_to_gephi(save=False)

# Selective edge pruning

In [7]:
tf_range = list(range(3, 16, 1))
h_exps, h_entropy = GtExp.load_sbm_exps(all_h, name="standard_5K", exp_type="h42_700_degreeCor", tf_range=tf_range)
h_entropy["Type"] = "Experiment"

### Loaded standard_int_healthy42_5K_3TF
### Loaded standard_int_healthy42_5K_4TF
### Loaded standard_int_healthy42_5K_5TF
### Loaded standard_int_healthy42_5K_6TF
### Loaded standard_int_healthy42_5K_7TF
### Loaded standard_int_healthy42_5K_8TF
### Loaded standard_int_healthy42_5K_9TF
### Loaded standard_int_healthy42_5K_10TF
### Loaded standard_int_healthy42_5K_11TF
### Loaded standard_int_healthy42_5K_12TF
### Loaded standard_int_healthy42_5K_13TF
### Loaded standard_int_healthy42_5K_14TF
### Loaded standard_int_healthy42_5K_15TF


In [8]:
%%capture
ctrl_exps, cmb_df = {}, pd.DataFrame()
for key in range(1, 11, 1):
    exps, entropy = GtExp.load_sbm_exps(h_ctrls[key], name="standard_5K", exp_type="sbm_ctrl42_standard_5K", tf_range=tf_range)
    entropy["Type"] = "hCtrl{}".format(key)
    cmb_df = pd.concat([cmb_df, entropy], axis=0)
    ctrl_exps[key] = {"entropy": entropy, "exps": exps}

cmb_df = pd.concat([cmb_df, h_entropy], axis=0)
cmb_df["Entropy_norm"] = (cmb_df["Entropy"] - cmb_df["Entropy"].min()) / (cmb_df["Entropy"].max() - cmb_df["Entropy"].min())
cmb_df["Entropy_log10"] = np.log10(cmb_df["Entropy"])


In [9]:
#### Find the network with top model
top_exp, color_map = [], {}
for exp in cmb_df["Type"].unique():
    sel_df = cmb_df[cmb_df["Type"] == exp]
    for tf in sel_df["TF"].unique():
        tst_df = sel_df[sel_df["TF"] == tf].sort_values(by="Entropy", ascending=False).values[0]
        top_exp.append(tst_df)

    color_map[exp] = px.colors.qualitative.Plotly[0]
    if exp == "Experiment":
        color_map[exp] = px.colors.qualitative.Plotly[2]

top_exp = pd.DataFrame(top_exp, columns=["Entropy", "TF", "Type", "Entropy_norm", "Entropy_log"])

#### Compute stats
stats_vals = []
for tf in top_exp["TF"].unique():
    sel_df = top_exp[top_exp["TF"] == tf]
    sel_df_ctrl = sel_df[sel_df["Type"].str.contains("hCtrl")]["Entropy"]
    sel_df_exp = sel_df[~sel_df["Type"].str.contains("hCtrl")]["Entropy"].values[0]
    stats_vals.append((sel_df_ctrl.median(), sel_df_ctrl.mean(), sel_df_ctrl.std(), tf, "Control"))
    stats_vals.append((sel_df_exp, sel_df_exp, 0, tf, "Experiment"))

In [10]:
top_df = pd.DataFrame(stats_vals, columns=["Median_Ent", "Mean_Ent", "Std_Ent", "TF", "Type"])

color_map = {"Experiment": px.colors.qualitative.G10[0], "Control": px.colors.qualitative.G10[1]}
fig = px.scatter(
    top_df,
    y="Mean_Ent",
    x="TF",
    error_y="Std_Ent",
    color="Type",
    title="Entropy evolution TFs 3-15. Standard 5K",
    height=600,
    color_discrete_map=color_map,
)

fig.update_layout(xaxis=dict(tickmode="array", tickvals=list(range(3, 16, 1))))
fig.update_layout(
    legend=dict(
        orientation="h",
        title="SBM",
        yanchor="bottom",
        y=0.87,
        xanchor="center",
        x=0.5,
        bgcolor="rgba(0,0,0,0)",
        font=dict(size=16, color="#003366"),
    ),
    title="",
    template="ggplot2",  # "ggplot2", "plotly_white"
    paper_bgcolor="rgba(0,0,0,0)",
    # plot_bgcolor="rgba(0,0,0,0)",
    xaxis=dict(
        tickfont=dict(size=16),
        title="Minimum degree for selected genes",
    ),
    yaxis=dict(
        tickfont=dict(size=16),
        title="Entropy",
    ),
    font=dict(size=16),
)
fig.update_traces(marker_size=8)
fig.update_traces(error_y=dict(width=8), selector=({"name": "Control"}))
fig.show()
save_fig(name="sbm_ent_sel_prun", fig=fig, base_path=figures_path, width=1200, height=600)

In [11]:
# Old figure with box plots
if False:
    fig = px.box(cmb_df, y="Entropy", x="TF", color="Type", title="Entropy evolution TFs 3-15. Standard 5K", height=600, color_discrete_map=color_map)

    fig.update_layout(xaxis=dict(tickmode="linear", tick0=3, dtick=1))
    fig.update_layout(xaxis=dict(tickmode="array", tickvals=list(range(3, 15, 1))))
    fig.update_xaxes(tickfont=dict(size=14), title_font=dict(size=16))
    fig.update_yaxes(tickfont=dict(size=14), title_font=dict(size=16))
    fig.show()

# Leiden and SBM comparisons 

## Modularity Score

In [15]:
def prep_leiden(exp: ExperimentSet, label="Experiment"):
    leid_stats = exp.comb_leiden_scores()
    leid_stats = leid_stats.loc[leid_stats["Modifier"] == "Standard"]
    leid_stats["Type"] = label

    sel_cols = ["Leiden Rank", "ModularityScore", "ModuleNum", "AvgModSize", "Modifier", "Type", "TF", "AvgModuleNum"]

    return leid_stats[sel_cols]


def tf_stats(ctrls_df: pd.DataFrame, exp_df: pd.DataFrame):
    for tf in ctrls_df["TF"].unique():
        sel_df = ctrls_df[ctrls_df["TF"] == tf]
        sel_df_exp = exp_df[exp_df["TF"] == tf]

        r_field = {"ModularityScore": "Mod", "AvgModuleNum": "ModNum", "AvgModSize": "ModSize"}
        for field in ["ModularityScore", "AvgModuleNum", "AvgModSize"]:
            ctrls_df.loc[ctrls_df["TF"] == tf, "{}_median".format(r_field[field])] = sel_df[field].median(axis=0)
            ctrls_df.loc[ctrls_df["TF"] == tf, "{}_mean".format(r_field[field])] = sel_df[field].mean(axis=0)
            ctrls_df.loc[ctrls_df["TF"] == tf, "{}_std".format(r_field[field])] = sel_df[field].std(axis=0)

            exp_df.loc[exp_df["TF"] == tf, "{}_median".format(r_field[field])] = sel_df_exp[field].median(axis=0)
            exp_df.loc[exp_df["TF"] == tf, "{}_mean".format(r_field[field])] = sel_df_exp[field].mean(axis=0)
            exp_df.loc[exp_df["TF"] == tf, "{}_std".format(r_field[field])] = sel_df_exp[field].std(axis=0)

    ctrls_df["Type"] = "Control"
    comb_leiden = pd.concat([ctrls_df, exp_df], axis=0)
    comb_leiden["TF"] = comb_leiden["TF"].astype(int)
    comb_leiden = comb_leiden[comb_leiden["TF"] <= 15]
    comb_leiden.sort_values(by="TF", ascending=True, inplace=True)
    comb_leiden["TF"] = comb_leiden["TF"].astype(str)

    return comb_leiden

In [16]:
leid_h = prep_leiden(all_h, label="Experiment")

leid_ctrls = []
for idx in range(1, 11, 1):
    df = prep_leiden(h_ctrls[idx], label="hCtrl{}".format(idx))
    leid_ctrls.append(df)

all_leid_ctrls = pd.concat(leid_ctrls, axis=0)
top_leid_ctrls = all_leid_ctrls[all_leid_ctrls["Leiden Rank"] == 0]

In [17]:
top_comb_leiden = tf_stats(top_leid_ctrls, leid_h[leid_h["Leiden Rank"] == 0])
fig = px.line(
    top_comb_leiden,
    x="TF",
    y="Mod_mean",
    error_y="Mod_std",
    markers=True,
    color="Type",
    height=600,
    title="Modularity scores for the best top Leiden Run.",
    color_discrete_map=color_map,
)
fig.update_layout(
    legend=dict(
        orientation="h",
        title="Leiden",
        yanchor="bottom",
        y=0.87,
        xanchor="center",
        x=0.5,
        bgcolor="rgba(0,0,0,0)",
        font=dict(size=16, color="#003366"),
    ),
    title="",
    template="ggplot2",  # "ggplot2", "plotly_white"
    paper_bgcolor="rgba(0,0,0,0)",
    # plot_bgcolor="rgba(0,0,0,0)",
    xaxis=dict(
        tickfont=dict(size=16),
        title="Minimum degree for selected genes",
    ),
    yaxis=dict(
        tickfont=dict(size=16),
        title="Modularity Score",
    ),
    font=dict(size=16),
)
fig.update_traces(line_width=5, marker_size=12, error_y=dict(width=5))

fig.show()
save_fig(name="leid_mod_sel_prun", fig=fig, base_path=figures_path, width=1200, height=600)

In [13]:
# top_leid_ctrls["TF"] = top_leid_ctrls["TF"].astype(int)
# top_leid_ctrls.sort_values(by="TF", ascending=True)
# x = list(top_leid_ctrls["TF"].astype(int))
# y_upper = list(top_leid_ctrls["Mod_mean"] + top_leid_ctrls["Mod_std"])
# y_lower = list(top_leid_ctrls["Mod_mean"] - top_leid_ctrls["Mod_std"])


# fig.add_trace(
#     go.Scatter(
#         x=x + x[::-1],  # x, then x reversed
#         y=y_upper + y_lower[::-1],  # upper, then lower reversed
#         fill="toself",
#         fillcolor="rgba(0,100,80,0.2)",
#         line=dict(color="rgba(255,255,255,0)"),
#         hoverinfo="skip",
#         showlegend=False,
#     )
# )

# fig.show()

In [14]:
leid_fig = px.line(
    top_comb_leiden,
    x="TF",
    y="ModNum_mean",
    error_y="ModNum_std",
    markers=True,
    color="Type",
    height=600,
    title="Community sizes for the best top Leiden Run.",
    color_discrete_map=color_map,
)
leid_fig.update_layout(
    legend=dict(
        orientation="h",
        title="Leiden",
        yanchor="bottom",
        y=1.1,
        xanchor="center",
        x=0.5,
        bgcolor="rgba(0,0,0,0)",
        font=dict(size=16, color="#003366"),
    ),
    title="",
    template="ggplot2",  # "ggplot2", "plotly_white"
    # paper_bgcolor="rgba(0,0,0,0)",
    # plot_bgcolor="rgba(0,0,0,0)",
    xaxis=dict(
        tickfont=dict(size=16),
        title="Minimum degree for selected genes",
    ),
    yaxis=dict(
        tickfont=dict(size=16),
        tickmode="linear",
        # tick0=3,
        # dtick=1,
        title="Number of communities",
    ),
    font=dict(size=16),
)
# leid_fig.update_traces(line_width=5, marker_size=12, error_y=dict(width=5))
# leid_fig.show()
save_fig(name="leid_comNum_sel_prun", fig=leid_fig, base_path=figures_path, width=1400, height=700)

## SBM - Community size

In [15]:
ctrl_sizes, h_sizes = [], []
for key, val in h_exps.items():
    results_df = val.get_gt_df(compute=False)
    h_sizes.append(((key, "Experiment", len(results_df["max_b"].unique()))))
    for idx in range(1, 11, 1):
        control_res = ctrl_exps[idx]["exps"][key].get_gt_df(compute=False)
        ctrl_sizes.append(((key, "hCtrl{}".format(idx), len(control_res["max_b"].unique()))))

ctrl_sizes = pd.DataFrame(ctrl_sizes, columns=["TF", "Type", "Com_size"])
ctrl_sizes["Type"] = "Control"
h_sizes = pd.DataFrame(h_sizes, columns=["TF", "Type", "Com_size"])

In [16]:
for tf in ctrl_sizes["TF"].unique():
    sel_df = ctrl_sizes[(ctrl_sizes["TF"] == tf)]
    sel_df_exp = h_sizes[h_sizes["TF"] == tf]

    ctrl_sizes.loc[ctrl_sizes["TF"] == tf, "Size_median"] = sel_df["Com_size"].median(axis=0)
    ctrl_sizes.loc[ctrl_sizes["TF"] == tf, "Size_mean"] = sel_df["Com_size"].mean(axis=0)
    ctrl_sizes.loc[ctrl_sizes["TF"] == tf, "Size_std"] = sel_df["Com_size"].std(axis=0)

    h_sizes.loc[h_sizes["TF"] == tf, "Size_median"] = sel_df_exp["Com_size"].median(axis=0)
    h_sizes.loc[h_sizes["TF"] == tf, "Size_mean"] = sel_df_exp["Com_size"].mean(axis=0)
    h_sizes.loc[h_sizes["TF"] == tf, "Size_std"] = 0

comb_df = pd.concat([ctrl_sizes, h_sizes], axis=0)

In [17]:
if True:
    sbm_fig = px.line(
        comb_df,
        x="TF",
        y="Size_mean",
        color="Type",
        error_y="Size_std",
        markers=True,
        title="SBM. Community sizes",
        height=600,
        color_discrete_map=color_map,
    )
    sbm_fig.update_layout(
        legend=dict(
            orientation="h",
            title="Type",
            yanchor="bottom",
            y=1.1,
            xanchor="center",
            x=0.5,
            bgcolor="rgba(0,0,0,0)",
            font=dict(size=16, color="#003366"),
        ),
        title="",
        template="ggplot2",  # "ggplot2", "plotly_white"
        # paper_bgcolor="rgba(0,0,0,0)",
        # plot_bgcolor="rgba(0,0,0,0)",
        xaxis=dict(
            tickfont=dict(size=16),
            title="Minimum degree for selected genes",
        ),
        yaxis=dict(
            tickfont=dict(size=16),
            tickmode="linear",
            # tick0=3,
            # dtick=1,
            title="Number of communities",
        ),
        font=dict(size=16),
    )
    # sbm_fig.update_traces(line_width=5, marker_size=12, error_y=dict(width=5))
    # sbm_fig.show()
    save_fig(name="sbm_comNum_sel_prun", fig=sbm_fig, base_path=figures_path, width=1200, height=600)

### SBM + Leiden Community size

In [18]:
sbm_fig = sbm_fig.add_traces(leid_fig.data)
sbm_fig = sbm_fig.update_layout(
    legend=dict(
        orientation="h",
        title="Type",
        yanchor="bottom",
        y=1.1,
        xanchor="center",
        x=0.5,
        bgcolor="rgba(0,0,0,0)",
        font=dict(size=16, color="#003366"),
    ),
    paper_bgcolor="rgba(0,0,0,0)",
    xaxis=dict(
        tickfont=dict(size=16),
        title="Minimum degree for selected genes",
    ),
    title="",
    template="ggplot2",
    yaxis=dict(
        tickmode="linear",
        tick0=10,
        dtick=2,
    ),
    showlegend=False,
    font=dict(size=16),
)
sbm_fig = sbm_fig.update_traces(line_width=5, marker_size=12, error_y=dict(width=5))
sbm_fig.show()
save_fig(name="sbm_Leiden_combNum", fig=sbm_fig, base_path=figures_path, width=1200, height=600)

# TF representation

In [19]:
def gene_sel_by_mev(exp, exp_label="", show_figs=True):
    all_exps = []
    for num_genes in range(25, 201, 25):
        exp_genes = []
        for modCon in exp.gt_modCons.values():
            top_100 = modCon.sort_values(by=["ModCon_{}_gt".format(exp.type)], ascending=False).index.values[:num_genes]
            exp_genes.extend(top_100)

        # add to the global exps
        sel_df = tcga_mutations_df[tcga_mutations_df.index.isin(exp_genes)]
        mut_0 = sel_df[sel_df["count"] > 0].shape[0]
        mut_5 = sel_df[sel_df["count"] >= 5].shape[0]
        mut_10 = sel_df[sel_df["count"] >= 10].shape[0]

        # tf exploration
        tf_used = set(exp_genes) & set(tf_list)
        tf_used_mut = sel_df[sel_df["count"] > 0]
        tf_used_mut = tf_used_mut[tf_used_mut.index.isin(list(tf_used))].shape[0]

        all_exps.append((len(exp_genes), mut_0, mut_5, mut_10, len(tf_used), tf_used_mut, str(num_genes)))

    genes_stats = pd.DataFrame(all_exps, columns=["#Genes", "#Mutated >0", "#Mutated >5", "#Mutated >10", "#TF", "#Mutated TF", "Exp"])

    metrics_cols = genes_stats.columns[:-1]
    remap_cols = {col: col.replace("#", "%") for col in metrics_cols}
    prct_df = pd.concat(
        [genes_stats["Exp"], genes_stats[metrics_cols].div(genes_stats["#Genes"].values, axis=0).multiply(100).rename(columns=remap_cols)], axis=1
    )

    if show_figs:
        # Proportion
        fig = px.bar(
            prct_df,
            x="Exp",
            y=list(remap_cols.values()),
            barmode="group",
            title="{}. Proportion of genes included when different #num selected by ModCon".format(exp_label),
            height=600,
            text_auto=True,
        )
        fig.show()

        # Numbers
        fig = px.bar(
            genes_stats,
            x="Exp",
            y=genes_stats.columns[:-1],
            barmode="group",
            title="{}. Number of genes when different #num selected by ModCon".format(exp_label),
            height=600,
        )
        fig.show()
    return genes_stats, prct_df, exp_genes


def tf_modCon_exps(exps, tf_range=None):
    tf_changes = pd.DataFrame()
    if tf_range is None:
        tf_range = range(3, 11, 1)
    for tf in tf_range:
        sel_exp = exps[str(tf)]
        genes_used, _, _ = gene_sel_by_mev(sel_exp, exp_label=str(tf), show_figs=False)

        remap_cols = {col: "{}TF_{}".format(tf, col) for col in ["#TF", "#Mutated TF", "Exp"]}
        dmy = genes_used[list(remap_cols.keys())].copy(deep=True)
        dmy["TF_edges"] = "{}".format(tf)
        tf_changes = pd.concat(
            [tf_changes, dmy],
            axis=0,
        )

    return tf_changes


# This is more for development
def plot_single_ctrl(ctrl_exps: dict, key=1):
    #### This works for only one Control select the ModCon - 100

    tf_changes_exp = tf_modCon_exps(h_exps, tf_range=list(list(range(3, 16, 1))))
    tf_changes_control = tf_modCon_exps(ctrl_exps[key]["exps"], tf_range=list(list(range(3, 16, 1))))

    tf_changes_exp["Type"] = "Experiment"
    tf_changes_control["Type"] = "Control"

    dmy_df = pd.concat([tf_changes_exp, tf_changes_control], axis=0)
    dmy_df = dmy_df[dmy_df["Exp"] == "100"]
    dmy_df["Used_prct"] = dmy_df["#TF"] / len(used_tf) * 100

    fig = px.line(dmy_df, x="TF_edges", y="Used_prct", color="Type", markers=True, title="% TF used in calculating the MEV", height=600)
    fig.update_layout(
        legend=dict(
            orientation="h",
            # title="Type",
            yanchor="bottom",
            y=0.18,
            xanchor="center",
            x=0.5,
            bgcolor="rgba(0,0,0,0)",
            font=dict(size=12),
        ),
    )
    fig.update_xaxes(title_text="Minimum degree for selected genes", tickfont=dict(size=14), title_font=dict(size=16))
    fig = fig.update_yaxes(title_text="% of genes included", range=[0, 105], tickfont=dict(size=14), title_font=dict(size=16))
    fig.show()


def worker(arg):
    obj, methname = arg[:2]
    _ = getattr(obj, methname)()
    return obj

In [20]:
results = pool.map(worker, ((exp, "get_ModCon") for exp in h_exps.values()))
h_exps = {exp.type.split("_")[-1].split("TF")[0]: exp for exp in results}

for key, exp in h_exps.items():
    sort_col = "ModCon_{}_gt".format(exp.type)
    exp.mevsMut, _ = exp.get_mevs(tpms=all_tum_tpms, modCon=exp.gt_modCons, sort_col=sort_col, num_genes=100, verbose=False)

In [21]:
# Healthy control for ModCon and MEV
### ModCon
for key in ctrl_exps.keys():
    print("### ModCon Control {}".format(key))
    results = pool.map(worker, ((exp, "get_ModCon") for exp in ctrl_exps[key]["exps"].values()))
    ctrl_exps[key]["exps"] = {exp.type.split("_")[-1].split("TF")[0]: exp for exp in results}

### MEV
for key in ctrl_exps.keys():
    print("### MEV Control {}".format(key))
    for key, exp in ctrl_exps[key]["exps"].items():
        sort_col = "ModCon_{}_gt".format(exp.type)
        exp.mevsMut, _ = exp.get_mevs(tpms=all_tum_tpms, modCon=exp.gt_modCons, sort_col=sort_col, num_genes=100, verbose=False)

### ModCon Control 1
### ModCon Control 2
### ModCon Control 3
### ModCon Control 4
### ModCon Control 5
### ModCon Control 6
### ModCon Control 7
### ModCon Control 8
### ModCon Control 9
### ModCon Control 10
### MEV Control 1
### MEV Control 2
### MEV Control 3
### MEV Control 4
### MEV Control 5
### MEV Control 6
### MEV Control 7
### MEV Control 8
### MEV Control 9
### MEV Control 10


In [22]:
# Control TF changes
used_tf = all_h.exps["standard_5K_3TF"].nodes_df["TF"][all_h.exps["standard_5K_3TF"].nodes_df["TF"] == 1].index.values

tf_chgs_all_ctrls = []
tf_range = list(list(range(3, 16, 1)))
for key, val in ctrl_exps.items():
    tf_chgs = tf_modCon_exps(val["exps"], tf_range=tf_range)
    tf_chgs = tf_chgs[tf_chgs["Exp"] == "100"]
    tf_chgs["Type"] = "hCtrl{}".format(key)
    tf_chgs["Used_prct"] = tf_chgs["#TF"] / len(used_tf) * 100
    # tf_chgs["Mut_prct"] = tf_chgs["#Mutated TF"] / len(used_tf) * 100 #used_tf which are mutated
    tf_chgs_all_ctrls.append(tf_chgs)

tf_chgs_all_ctrls = pd.concat(tf_chgs_all_ctrls, axis=0)

# Generate the Exp Changes too
tf_changes_exp = tf_modCon_exps(h_exps, tf_range=tf_range)

In [23]:
# Process the information and get some stats

## Control
all_tf_chgs_stats = []
for tf in tf_chgs_all_ctrls["TF_edges"].unique():
    sel_df = tf_chgs_all_ctrls[tf_chgs_all_ctrls["TF_edges"] == tf]
    all_tf_chgs_stats.append(
        [
            tf,
            sel_df["Exp"].values[0],
            sel_df["Used_prct"].median(axis=0),
            sel_df["Used_prct"].mean(axis=0),
            sel_df["Used_prct"].std(axis=0),
            sel_df["#TF"].median(axis=0),
            sel_df["#TF"].mean(axis=0),
            sel_df["#TF"].std(axis=0),
            sel_df["#Mutated TF"].median(axis=0),
            sel_df["#Mutated TF"].mean(axis=0),
            sel_df["#Mutated TF"].std(axis=0),
        ]
    )


tf_chgs_stats_df = pd.DataFrame(
    all_tf_chgs_stats,
    columns=["TF_edges", "Exp", "Prct_median", "Prct_mean", "Prct_std", "#TF_median", "#TF_mean", "#TF_std", "#Mutated median", "Mutated mean", "Mutated std"],
)
tf_chgs_stats_df["Type"] = "Control"

## Experiment
tf_changes_exp = tf_changes_exp[tf_changes_exp["Exp"] == "100"]
tf_changes_exp["Used_prct"] = tf_changes_exp["#TF"] / len(used_tf) * 100
tf_changes_exp["Prct_median"] = tf_changes_exp["Used_prct"]
tf_changes_exp["Prct_mean"] = tf_changes_exp["Used_prct"]
tf_changes_exp["Prct_std"] = 0
tf_changes_exp["Type"] = "Experiment"
# tf_changes_exp["Mut_prct"] = tf_changes_exp["#Mutated TF"] / len(used_tf) * 100

## Combine the two
comb_chgs = pd.concat([tf_chgs_stats_df, tf_changes_exp], axis=0)

In [24]:
fig = px.line(
    comb_chgs,
    x="TF_edges",
    y="Prct_mean",
    error_y="Prct_std",
    color="Type",
    color_discrete_map=color_map,
    markers=True,
    title="% TF used in calculating the MEV",
    height=600,
)
fig.update_layout(
    legend=dict(
        orientation="h",
        # title="Type",
        yanchor="bottom",
        y=0.18,
        xanchor="center",
        x=0.5,
        bgcolor="rgba(0,0,0,0)",
        font=dict(size=16),
    ),
    title="",
    template="ggplot2",  # "ggplot2", "plotly_white"
    paper_bgcolor="rgba(0,0,0,0)",
    # plot_bgcolor="rgba(0,0,0,0)",
    xaxis=dict(
        tickfont=dict(size=16),
        title_font=dict(
            size=16,
        ),
        title="Minimum degree for selected genes",
    ),
    yaxis=dict(
        tickfont=dict(size=16),
        title_font=dict(
            size=16,
        ),
        title="% of genes included",
        range=[0, 105],
    ),
    font=dict(size=16),
)
# fig = fig.update_yaxes(title_text="% of genes included", range=[0, 105], tickfont=dict(size=14), title_font=dict(size=16))
fig.update_traces(line_width=5, marker_size=12, error_y=dict(width=5))
save_fig(name="ctrls_min_dig_mev", fig=fig, base_path=figures_path, width=1200, height=600)
fig.show()

In [ ]:
# for development
if False:
    for key in range(1, 3, 1):
        display(plot_single_ctrl(ctrl_exps, key=key))

## Find common genes in TFs

In [ ]:
for key, val in ctrl_exps.items():
    tf_chgs = tf_modCon_exps(val["exps"], tf_range=tf_range)

In [98]:
num_genes = 100
all_ctrls = {}
for ctrl_exp in ctrl_exps.values():
    all_exps = {}
    for exp in ctrl_exp["exps"].values():
        exp_genes = []
        for modCon in exp.gt_modCons.values():
            top_100 = modCon.sort_values(by=["ModCon_{}_gt".format(exp.type)], ascending=False).index.values[:num_genes]
            exp_genes.extend(top_100)

        # tf exploration
        tf_used = set(exp_genes) & set(tf_list)
        if all_exps:
            all_exps = tf_used & all_exps
        else:
            all_exps = tf_used

    if all_ctrls:
        all_ctrls = all_exps & all_ctrls
    else:
        all_ctrls = all_exps

# To save the list
if True:
    dmy_df = pd.DataFrame(index=list(all_ctrls))
    dmy_df["mut_count"] = tcga_mutations_df["count"]
    sel_tum_df = all_tum_tpms.loc[all_tum_tpms.index.isin(list(all_ctrls))]
    sel_h_df = exp.tpm_df.loc[exp.tpm_df.index.isin(list(all_ctrls))]

    dmy_df["tum_median_expression"] = sel_tum_df.median(axis=1)
    dmy_df["healthy_median_expression"] = sel_h_df.median(axis=1)

    dmy_df["tum_mean_expression"] = sel_tum_df.mean(axis=1)
    dmy_df["healthy_mean_expression"] = sel_h_df.mean(axis=1)

    dmy_df["tum_std_expression"] = sel_tum_df.std(axis=1)
    dmy_df["healthy_std_expression"] = sel_h_df.std(axis=1)

    # Some interesting info
    print("TFs mutated but not expressed: ")
    display(dmy_df[dmy_df["tum_median_expression"].isna()])

    dmy_df.fillna(0, inplace=True)
    dmy_df.to_csv(data_base + "tf_ctrl.csv")

TFs mutated but not expressed: 


,mut_count,tum_median_expression,healthy_median_expression
BNC1,7.0,NaN,13.012142
HES2,1.0,NaN,20.831714


In [102]:
dmy_df["mut_count"] = tcga_mutations_df["count"]
sel_tum_df = all_tum_tpms.loc[all_tum_tpms.index.isin(list(all_ctrls))]
sel_h_df = exp.tpm_df.loc[exp.tpm_df.index.isin(list(all_ctrls))]

dmy_df["tum_median_expression"] = sel_tum_df.median(axis=1)
dmy_df["healthy_median_expression"] = sel_h_df.median(axis=1)

dmy_df["tum_mean_expression"] = sel_tum_df.mean(axis=1)
dmy_df["healthy_mean_expression"] = sel_h_df.mean(axis=1)

dmy_df["tum_std_expression"] = sel_tum_df.std(axis=1)
dmy_df["healthy_std_expression"] = sel_h_df.std(axis=1)

# Some interesting info
print("TFs mutated but not expressed: ")
display(dmy_df[dmy_df["tum_median_expression"].isna()])

dmy_df.fillna(0, inplace=True)
dmy_df.to_csv(data_base + "tf_ctrl.csv")

TFs mutated but not expressed: 


,mut_count,tum_median_expression,healthy_median_expression,tum_mean_expression,healthy_mean_expression,tum_std_expression,healthy_std_expression
BNC1,7.0,NaN,13.012142,NaN,27.664774,NaN,37.200816
HES2,1.0,NaN,20.831714,NaN,43.486837,NaN,55.283490


In [168]:
tst_df = dmy_df.reset_index(names="gene")
tst_df = tst_df[~tst_df["gene"].isin(["ELF3", "JUNB"])]
fig = px.scatter(
    tst_df,
    x="tum_mean_expression",
    y="healthy_mean_expression",
    # error_x="tum_std_expression",
    # error_y="healthy_std_expression",
    # text="gene",
    color="mut_count",
    size="mut_count",
    hover_data="gene",
    height=700,
)
# Get the top 10 genes with highest healthy median expression
top_genes_h = tst_df.nlargest(10, "healthy_median_expression")["gene"].tolist()
top_genes_t = tst_df.nlargest(10, "tum_median_expression")["gene"].tolist()
top_genes_m = tst_df.nlargest(10, "mut_count")["gene"].tolist()

top_genes = list(set(top_genes_h + top_genes_t + top_genes_m))
top_genes.extend(["FOSL1", "FOXQ1", "ERF", "MYCL", "STAT2", "IRF7", "CIC", "MBD6", "ZNF750", "ZNF513"])
chg_genes = ["KMT2A", "SPEN", "MEDCOM", "ERF", "AHR", "STAT2", "MBD6", "ZNF513", "TRERF1"]
# Add text annotation for the top 10 genes
for idx, gene in enumerate(top_genes):
    x = tst_df.loc[tst_df["gene"] == gene, "tum_mean_expression"].values[0]
    y = tst_df.loc[tst_df["gene"] == gene, "healthy_mean_expression"].values[0]
    ay = -30
    if idx % 2:
        ax = 5
    else:
        ax = -5

    if gene in chg_genes:
        ax = -ax
        ay = -ay

    fig.add_annotation(x=x, y=y, text=gene, showarrow=True, arrowhead=1, ax=ax, ay=ay)

save_fig(name="ctrlTFs", fig=fig, width=1600, height=700, base_path=figures_path)

# Visualise SBM

In [ ]:
def callback(self, g, keyval, picked, pos, vprops, eprops):
    # label
    if type(picked) != gt.VertexPropertyMap:
        print("{}. Comm {}. Mut {}. Idx {}".format(g.vp.gene[picked], g.vp.max_b[picked], g.vp.mut_count[picked], picked))

In [ ]:
gt_exp = h_exps["6"]
gt_exp.layout()
gt_exp.add_vp(mut_df=tcga_mutations_df)
pos = gt_exp.pos
gt_g = gt_exp.gt_g
pv = gt_exp.states[0]["pv"]
pmode = gt_exp.states[0]["pmode"]

gt_g = gt_exp.gt_g
state = gt.BlockState(gt_g, recs=[gt_g.ep.weight], rec_types=["real-exponential"])

In [52]:
# interactive session
if False:
    state.draw(
        pos=pos,
        vertex_shape="pie",
        vertex_pie_fractions=pv,
        inline=False,
        nodeFirst=True,
        # aspect=0.4,
        ink_scale=0.4,
        vertex_text=gt_g.vp.gene,
        display_props=gt_g.vp.gene,
        output_size=(2000, 2000),  # debugging
        #### Nodes properties
        # vertex_size=gt.prop_to_size(gt_g.vp.mut_count, 1, 20, power=1),
        vertex_font_size=gt.prop_to_size(gt_g.vp.mut_count, 10, 14, power=1.0),  # 5, 30 for notebook  viz
        ##### edges properties
        # edge_color=gt.prop_to_size(gt_g.ep.weight, power=1, log=True),
        edge_pen_width=gt.prop_to_size(gt_g.ep.weight, 0, 3, power=0.1),
        # edge_gradient=[],
        # vorder=gt_g.vp.mut_count,
        key_press_callback=callback,
        # text_out_width=0.1,
    )

ENSG00000225886. Comm 23. Mut 0. Idx 1236
IFI27. Comm 23. Mut 0. Idx 1569
IFI27. Comm 23. Mut 0. Idx 1569
IFI27. Comm 23. Mut 0. Idx 1569
S100A4. Comm 20. Mut 1. Idx 3957
KANSL1. Comm 15. Mut 23. Idx 4247
SCAF1. Comm 15. Mut 7. Idx 145


In [ ]:
# Config to save it as PDF
state.draw(
    pos=pos,
    vertex_shape="pie",
    vertex_pie_fractions=pv,
    inline=False,
    vertex_text=gt_g.vp.gene,
    display_props=gt_g.vp.gene,
    output_size=(8000, 8000),  # good to visualised in Notebook
    #### Nodes properties
    vertex_size=gt.prop_to_size(gt_g.vp.mut_count, 1, 20, power=1),
    vertex_font_size=gt.prop_to_size(gt_g.vp.mut_count, 14, 24, power=1.0), 
    ##### edges properties
    edge_pen_width=gt.prop_to_size(gt_g.ep.weight, 0, 1, power=0.1),
    key_press_callback=callback,
    output=figures_path + "sbm_h_tf6_5K.pdf",
)

## Clustering analysis

In [228]:
# print(list(gt_exp.tpm_df.sort_index().index.values))
ensg_genes = gt_exp.tpm_df.reset_index()[gt_exp.tpm_df.reset_index()["genes"].str.contains("ENSG")].set_index("genes")
# ensg_genes["mut_count"] = tcga_mutations_df["count"]

# Go enrichment

In [9]:
from gseapy.plot import dualplot
import gseapy as gp

In [39]:
def get_pathways(com, exp, gt_graph: False, output_dir=None):
    if gt_graph:
        g_com = exp.gt_modCons[com].index.values
    else:
        g_com = exp.modCons[com].index.values

    enrich_r = gp.enrichr(
        gene_list=list(g_com),  # it needs to be a list
        gene_sets=["Reactome_2022", "Panther_2016"],
        organism="Human",
        top_term=20,
        outdir=output_dir,
        no_plot=True,
        expName="Comm_{}".format(com),
    )

    return enrich_r, list(g_com)


def process_enrichr(enrich_r):
    dmy = enrich_r.results.copy(deep=True)
    dmy = pd.concat([dmy, dmy["Overlap"].str.split("/", expand=True).rename(columns={0: "Matched", 1: "Ref"})], axis=1)
    dmy["Ratio"] = (dmy["Matched"].astype(int) / dmy["Ref"].astype(int)).round(3)
    reorder_cols = [
        "Term",
        "Overlap",
        "Ratio",
        "P-value",
        "Adjusted P-value",
        "Genes",
        "Matched",
        "Ref",
        "Old P-value",
        "Old Adjusted P-value",
        "Odds Ratio",
        "Gene_set",
    ]
    dmy = dmy[reorder_cols]

    return dmy


def plot_enrich(filtered_df, com, exp, savePath=None):
    title_dot = "enrichment for {} in {}".format(com, exp.type)
    top_term = 10
    cutoff = 0.05
    figsize = (3, 3)

    if not os.path.exists(savePath):
        os.makedirs(savePath)

    ax_dot = dualplot(
        filtered_df,
        column="Adjusted P-value",
        columnBar="Ratio",
        x="Gene_set",  # set x axis, so you could do a multi-sample/library comparsion
        size=12,
        top_term=top_term,
        figsize=figsize,
        title=title_dot,
        xticklabels_rot=45,  # rotate xtick labels
        show_ring=True,  # set to False to revmove outer ring
        marker="o",
        cutoff=cutoff,
        cutoffBar=1.0,  # needs to be able to show up to 100% match
        colorBar=["darkred", "darkblue"],
        group="Gene_set",
        ofname="{}/Com_{}_dualPlot.png".format(savePath, com) if savePath else None,
    )

    return ax_dot


def get_sig_pathways(pathways, df, gene_set="Reactome_2022", sort_cols=["Adjusted P-value", "Ratio"], ascending=True):
    dmy = df.loc[df["Gene_set"] == gene_set].sort_values(by=sort_cols, ascending=ascending).head(10)
    for _, row in dmy.iterrows():
        key_term = row["Term"][: row["Term"].find("R-")]
        key_term = key_term[: key_term.find("Homo") :]
        key_term = key_term.replace("pathway", "")

        if gene_set == "Reactome_2022":
            pathways["{}_R".format(key_term)] = '["{}"]'.format('","'.join(row["Genes"].split(";")))
            # pathways["{}_R".format(key_term)] = row["Genes"].split(";")
        else:
            pathways["{}_P".format(key_term)] = '["{}"]'.format('","'.join(row["Genes"].split(";")))
            # pathways["{}_P".format(key_term)] = row["Genes"].split(";")

    return pathways

In [11]:
go_lib = {
    "cell": "GO_Cellular_Component_2023",  # cellular component
    "process": "GO_Biological_Process_2023",  # biological process
    "molecular": "GO_Molecular_Function_2023",  # molecular function
}
pathways = {
    "reactome": "Reactome_2022",
    "panther": "Panther_2016",
}

In [12]:
# To be used only when single network is used
gt_exp = h_exps["6"]
sort_col = "ModCon_{}_gt".format(gt_exp.type)
gt_exp.add_vp()

gt_modCon = gt_exp.get_ModCon()
gt_exp.mevsMut, _ = gt_exp.get_mevs(tpms=all_tum_tpms, modCon=gt_modCon, sort_col=sort_col, num_genes=100, verbose=False)

In [13]:
# gt_exp = h_exps["6"] #this to be used when all experiments are loaded
output_dir = "{}/GeneOntology/{}/".format(figures_path, "6TF", gt_exp.type)

enrich_objs = {}
for com in gt_exp.gt_modCons.keys():
    enrich_r, genes = get_pathways(com=com, exp=gt_exp, gt_graph=True, output_dir=None)
    enrich_df = process_enrichr(enrich_r)

    enrich_objs[com] = {"enrich_r": enrich_r, "df": enrich_df, "genes": genes}

    filtered_df = enrich_df[enrich_df["Adjusted P-value"] < 0.05]
    if output_dir != None:
        if filtered_df.shape[0]:
            fig = plot_enrich(filtered_df, com=int(com), exp=gt_exp, savePath=output_dir)

        else:
            print("Comm {}. No (adujsted) significant p-values.".format(com))

Comm 14.0. No (adujsted) significant p-values.
Comm 17.0. No (adujsted) significant p-values.
Comm 20.0. No (adujsted) significant p-values.
Comm 22.0. No (adujsted) significant p-values.
Comm 26.0. No (adujsted) significant p-values.
Comm 30.0. No (adujsted) significant p-values.


In [14]:
import ast

filters = {}
com = 23
exp_enr = enrich_objs[com]
enrich = exp_enr["enrich_r"]
df = exp_enr["df"]
f_df = df[df["Adjusted P-value"] < 0.05].copy(deep=True)

pathways = get_sig_pathways({}, f_df, gene_set="Reactome_2022", sort_cols=["Ratio"], ascending=False)
pathways = get_sig_pathways(pathways, f_df, gene_set="Panther_2016", sort_cols=["Ratio"], ascending=False)

pathways = get_sig_pathways(pathways, f_df, gene_set="Reactome_2022", sort_cols=["Adjusted P-value"])
pathways = get_sig_pathways(pathways, f_df, gene_set="Panther_2016", sort_cols=["Adjusted P-value"])
filters[str(com)] = pathways

# easy to put to gephi
converted_dict = {key: ast.literal_eval(value) for key, value in filters[str(com)].items()}
for key, value in converted_dict.items():
    print(f'"{key}": {value},')

"TP53 Regulates Transcription Of Death Receptors And Ligands_R": ['TNFRSF10C', 'TP53', 'TNFRSF10D'],


### Export SBM to Gephi

In [2]:
results_df = gt_exp.get_stable_genes(state_idx=0, prob_th=0.75)
results_df["stable_gene_num"] = 0
results_df.loc[results_df["stable_gene"] == "wobble", "stable_gene_num"] = 1
results_df["stable_gene"].value_counts()

gt_exp.export_to_gephi(save=False, sbm_df=results_df, state_idx=0)

NameError: name 'gt_exp' is not defined

In [1]:
results_df

NameError: name 'results_df' is not defined

## Partition Prob

In [16]:
gt_exp.plot_partition_prob()

# Clustering analysis

In [40]:
gt_exps = {}
for key in ["3", "6", "10"]:
    dmy_exp = h_exps[key]
    sort_col = "ModCon_{}_gt".format(dmy_exp.type)
    gt_modCon = dmy_exp.get_ModCon()
    dmy_exp.mevsMut, _ = dmy_exp.get_mevs(tpms=all_tum_tpms, modCon=gt_modCon, sort_col=sort_col, num_genes=100, verbose=False)

    cs_exp, fig_std, metrics = gh.run_clusters(dmy_exp, label="exp_tf{}".format(key), show_figs=True)

    gt_exps[key] = {"exp": dmy_exp, "cs": cs_exp}

comb_cs = pd.concat([gt_exps["3"]["cs"], gt_exps["6"]["cs"], gt_exps["10"]["cs"], vu_output], axis=1).dropna()

# for single experiment
if False:
    tf = 6
    gt_exp = h_exps["6"]
    sort_col = "ModCon_{}_gt".format(gt_exp.type)
    gt_modCon = gt_exp.get_ModCon()
    gt_exp.mevsMut, _ = gt_exp.get_mevs(tpms=all_tum_tpms, modCon=gt_modCon, sort_col=sort_col, num_genes=100, verbose=False)

Variation per principal component [0.55728674 0.27211008] and the sum 82.94%
Variation per principal component [0.55226687 0.28018675] and the sum 83.25%
Variation per principal component [0.55491689 0.27757334] and the sum 83.25%


In [41]:
cluster_model, cs = "RawKMeans", 6
col_exp_3 = "{}_CS_{}_{}_tf{}".format(cluster_model, cs, "exp", 3)
col_exp_6 = "{}_CS_{}_{}_tf{}".format(cluster_model, cs, "exp", 6)
col_exp_10 = "{}_CS_{}_{}_tf{}".format(cluster_model, cs, "exp", 10)


cols = ["TCGA408_classifier", "KMeans_labels_6"] + [col_exp_3, col_exp_6, col_exp_10] + ["consensus"]
rename_cols = {
    cols[0]: "TCGA",
    cols[1]: "CA + IFNg",
    cols[2]: "SBM_3TF",
    cols[3]: "SBM_6TF",
    cols[4]: "SBM_10TF",
    cols[5]: "Consensus",
}

meta, fig = sky.main(df=comb_cs.rename(columns=rename_cols), reorder_cols=list(rename_cols.values()), title="", retMeta=True)

fig.update_layout(
    title="",
    template="ggplot2",  # "ggplot2", "plotly_white"
    font=dict(size=16),
)
fig.show()
save_fig(name="Sankey_comp_SBM_5K", fig=fig, base_path=figures_path, width=1200, height=600)

## Survival

In [284]:
def prep_survival(df, cs_num=6, tf=10, cs_model="RawKMeans", label="SBM"):
    model = "{}_CS_{}_exp_tf{}".format(cs_model, cs_num, tf)

    colors_net = px.colors.qualitative.G10
    for idx, val in enumerate(df[model].unique()):
        color_map[val] = colors_net[idx]

    df[model] = df[model].astype(str)
    fig = survival_plot(df.drop(columns=["days_to_last_follow_up", "days_to_death"]), vu_output, classifier=model, color_map=color_map)
    fig = fig.update_layout(title="{}. Survival analysis for {}".format(label, model))

    return fig


tcga_metadata = pd.read_csv(data_base + "TCGA_metadata.tsv", sep="\t", index_col="Sample")

In [286]:
fig = prep_survival(comb_cs, cs_num=6, tf=6, cs_model="RawKMeans", label="SBM")
# save_fig(name="Survival_plot_reward", fig=fig, base_path=figures_path, width=1400, height=600)

fig.update_layout(
    legend=dict(
        orientation="h",
        title="Network subtype",
        yanchor="bottom",
        y=0.1,
        xanchor="center",
        x=0.5,
        bgcolor="rgba(0,0,0,0)",
        font=dict(size=16, color="#003366"),
    ),
    title="",
    template="ggplot2",  # "ggplot2", "plotly_white"
    paper_bgcolor="rgba(0,0,0,0)",
    # plot_bgcolor="rgba(0,0,0,0)",
    xaxis=dict(tickfont=dict(size=16)),
    yaxis=dict(tickfont=dict(size=16)),
    font=dict(size=16),
)
fig.show()
save_fig(name="Survival_SBM_5K_6TF", fig=fig, base_path=figures_path, width=1000, height=600)

In [120]:
cluster_model = "RawKMeans_CS_6_exp_tf6"

colors_ref = px.colors.qualitative.Prism
color_map = {
    "LumP": colors_ref[0],
    "Lum Inf/Ns": colors_ref[1],
    "High IFNG": colors_ref[2],
    "Low IFNG": colors_ref[3],
    "Med IFNG": colors_ref[4],
    "Ne": colors_ref[5],
}

select_labels_1 = [1.0, 0.0, 5.0]
select_labels_2 = ["High IFNG", "Med IFNG", "Low IFNG"]

select_labels_1 = None
select_labels_2 = None

colors_net = px.colors.qualitative.Plotly
comb_cs[cluster_model] = comb_cs[cluster_model].astype(str)
for idx, val in enumerate(comb_cs[cluster_model].unique()):
    color_map[val] = colors_net[idx]

fig = survival_comp(
    comb_cs.drop(columns=["days_to_last_follow_up", "days_to_death"]),
    vu_output,
    classifier_1=cluster_model,
    classifier_2="KMeans_labels_6",
    selected_labels_1=select_labels_1,
    selected_labels_2=select_labels_2,
    color_map=color_map,
)
fig = fig.update_layout(title="Survival analysis {}".format("VU + in-situ"))
fig.update_layout(height=700)
# save_fig(name="Survival_plot_reward", fig=fig, base_path=figures_path, width=1400, height=600)

### Significance

In [24]:
from lifelines import KaplanMeierFitter
from lifelines.statistics import multivariate_logrank_test
from lifelines.utils import restricted_mean_survival_time
from lifelines.plotting import rmst_plot

In [93]:
def survival_sig(df, model):
    df = df.reset_index().rename(columns={"index": "Sample"}).copy(deep=True)
    classifier = model

    dmy = df[["days_to_last_follow_up", "days_to_death", classifier]].replace("--", 0).astype(int)
    dmy["last_contact"] = dmy[["days_to_last_follow_up", "days_to_death"]].max(axis=1).div(30)

    labels = list(df[model].unique())
    dmy = dmy[dmy[classifier].isin(labels)]

    results = multivariate_logrank_test(dmy["last_contact"], dmy[classifier], dmy["days_to_death"])
    display(results.print_summary())
    print("{0:.10f}".format(results.p_value))

In [95]:
comb_cs[cluster_model] = comb_cs[cluster_model].astype(int)
survival_sig(comb_cs, model=cluster_model)

[0, 1, 5, 2, 3, 4]


<lifelines.StatisticalResult: multivariate_logrank_test>
               t_0 = -1
 null_distribution = chi squared
degrees_of_freedom = 5
         test_name = multivariate_logrank_test

---
 test_statistic    p  -log2(p)
          13.52 0.02      5.72

None

0.0189304997
